<a href="https://colab.research.google.com/github/skim7871/CapstoneC57/blob/exp2/RISParser_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials 
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [10]:
!pip install nbdime


     |████████████████████████████████| 4.3MB 5.1MB/s 
     |████████████████████████████████| 460kB 42.0MB/s 
     |████████████████████████████████| 71kB 22.8MB/s 


In [13]:
!pip install -U -q RISparser

import os
from pprint import pprint
from RISparser import readris
from RISparser.config import TAG_KEY_MAPPING
from RISparser.config import LIST_TYPE_TAGS
#pprint(LIST_TYPE_TAGS)
#pprint(TAG_KEY_MAPPING)      

file_id = '1sXx2JXt3PDoWZk7Q660tMXDIo8tebtM3' 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('test.ris')
filepath = 'test.ris'

#file_id = '1y4cuCSGoGUVjfozjTFrHkEWrgUGPnvxV'
#downloaded = drive.CreateFile({'id': file_id})
#downloaded.GetContentFile('records (1).ris')
#filepath = 'records (1).ris'

mapping = TAG_KEY_MAPPING
mapping["SP"] = "pages_this_is_my_fun"
with open(filepath, 'r') as bibliography_file:
    entries = list(readris(bibliography_file, mapping=mapping))
    pprint(entries[0])
    #pprint(sorted(entries[0].keys()))
    #entries = readris(bibliography_file)
    #for entry in entries:      
    #  print(entry['primary_title'])
            


{'alternate_title3': 'Journal of Clinical Anesthesia',
 'author_address': 'H. Ueshima, Department of Anesthesiology, Showa University '
                   'Hospital, Hatanodai Shinagawa-ku, Tokyo, Japan',
 'custom5': '31176261',
 'doi': '10.1016/j.jclinane.2019.06.008',
 'file_attachments2': 'http://dx.doi.org/10.1016/j.jclinane.2019.06.008',
 'first_authors': ['Ueshima H.'],
 'issn': '1873-4529',
 'journal_name': 'J. Clin. Anesth.',
 'keywords': ['analgesic agent'],
 'language': 'English',
 'name_of_database': 'Embase',
 'note': '(Ueshima H., ueshimhi@med.showa-u.ac.jp; Hiroshi O.) Department of '
         'Anesthesiology, Showa University Hospital, Tokyo, Japan',
 'pages_this_is_my_fun': '7',
 'primary_title': 'Thoracoabdominal nerves block through the perichondral '
                  'approach for effective perioperative analgesia during upper '
                  'abdominal surgery',
 'publication_year': '2020',
 'type_of_reference': 'JOUR',
 'type_of_work': 'Letter',
 'unknown_tag'

In [15]:
import re
ALLOWED_TAGS = {"T1" : "Primary Title",
                "AB" : "Abstract"}
REFERENCE_TYPES = {"ABST" : "Abstract"}
class RIS:
    """ RIS file structure """
    def __init__(self, in_file=None):
        """ Initialize and parse input """
        self.records = []
        if in_file:
            self.parse(in_file)
    def parse(self, in_file):
        """ Parse input file """
        self.current_tag = None
        self.current_record = None
        prog = re.compile("^([A-Z][A-Z0-9]) *- *(.*)")
        lines = []
        # Eliminate blank lines
        for line in in_file:
            line = line.strip()
            if len(line) > 0:
                lines.append(line)
        for line in lines:
            if "1." in line:
              continue
            elif "2." in line:
              continue  
            match = prog.match(line)
            if match:
                tag = match.groups()[0]
                field = match.groups()[1]
                self.process_field(tag, field)
            else:
                raise ValueError(line)
    def process_field(self, tag, field):
        """ Process RIS file field """
        if tag == "TY":
            self.current_record = {tag: field}
        elif tag == "ER":
            self.records.append(self.current_record)
            self.current_record = None
        elif tag in ["AU", "AD", "KW", "N1"]:
            if tag in self.current_record:
                self.current_record[tag].append(field)
            else:
                self.current_record[tag] = [field]        
        else:
            if not tag in self.current_record:
                self.current_record[tag] = field
            else:
                error_str = "Duplicate tag: %s" % tag
                raise ValueError(error_str)

                
def main():
  """ Test the code """
  import pprint
  with open(filepath, "rt") as ris_file:
    ris = RIS(ris_file)
    pp = pprint.PrettyPrinter()
    pp.pprint(ris.records)  

        
main()                

[{'A1': 'Ueshima H.',
  'AD': ['H. Ueshima, Department of Anesthesiology, Showa University Hospital, '
         'Hatanodai Shinagawa-ku, Tokyo, Japan'],
  'C5': '31176261',
  'DB': 'Embase',
  'DO': '10.1016/j.jclinane.2019.06.008',
  'JF': 'Journal of Clinical Anesthesia',
  'JO': 'J. Clin. Anesth.',
  'KW': ['analgesic agent'],
  'L2': 'http://dx.doi.org/10.1016/j.jclinane.2019.06.008',
  'LA': 'English',
  'LK': 'http://za2uf4ps7f.search.serialssolutions.com/?sid=EMBASE&issn=18734529&id=doi:10.1016%2Fj.jclinane.2019.06.008&atitle=Thoracoabdominal+nerves+block+through+the+perichondral+approach+for+effective+perioperative+analgesia+during+upper+abdominal+surgery&stitle=J.+Clin.+Anesth.&title=Journal+of+Clinical+Anesthesia&volume=59&issue=&spage=7&epage=&aulast=Ueshima&aufirst=Hironobu&auinit=H.&aufull=Ueshima+H.&coden=JCLBE&isbn=&pages=7-&date=2020&auinit1=H&auinitm=',
  'M1': '(Ueshima H., ueshimhi@med.showa-u.ac.jp; Hiroshi O.) Department of '
        'Anesthesiology, Showa Universi